<a href="https://colab.research.google.com/github/Buterbrod/Deep-Learning-School-2021/blob/main/%D0%9F%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BE%D1%82%D1%82%D0%BE%D0%BA%D0%B0_%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_(%D0%BE%D1%81%D0%B5%D0%BD%D1%8C_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание оттока пользователей (весна 2021)

Это соревнование является домашним заданием к 3 модулю продвинутого потока DLS (весна 2021)
- https://www.kaggle.com/c/advanced-dls-spring-2021/overview

----

Вам предстоит **научиться моделировать отток клиентов телеком компании**. 

Эта задача очень важна на практике и алгоритмы для ее решения используются в реальных телеком компаниях, ведь если мы знаем, что клиент собирается уйти от нас, то мы попытаться удержать его, предложив какие-то бонусы.

**Дано:**

На странице с данными вы можете загрузить три файла:
- train.csv: файл с обучающей выборкой (содержит колонки признаков и целевую переменную)
- test.csv: файл с тестовой выборкой (содержит только колонки признаков).
- submission.csv: пример файла с предскзааниями

**Требуется:**

В качестве ответа вам необходимо загрузить предсказания вероятностей ухода клиентов для test.csv.

Названия признаков и целевой пременной есть в baseline ноутбуке. (baseline - базовое решение, с которым обычно сравниают более продвинутые алгоритмы.)

**Решение:**

## Шаг 1. Подключить библиотеки


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Шаг 2. Подключить Kaggle к Colab и забрать файлы соревнования



Документация:
- [kaggle-api](https://github.com/Kaggle/kaggle-api)
- [инструкция из интернета](https://askdev.ru/q/ispolzovanie-naborov-dannyh-kaggle-v-google-colab-581252)

**Предварительные шаги:**
- зайти на https://www.kaggle.com
- войти в account
- API, Create New API Token
- сохранить JSON на диск для дальнейшего подключения

In [ ]:
# Запустить и выбрать kaggle.json, ранее выгруженный файл с Kaggle account страницы
from google.colab import files
files.upload()
None

Saving kaggle.json to kaggle (2).json


In [ ]:
# Убедимся, что kaggle.json файл загрузился
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Oct  6 21:22 kaggle.json


In [57]:
# Установим клиент Kaggle API
!pip install -q kaggle

In [58]:
# Клиент Kaggle API ожидает файл kaggle.json в ~/.kaggle
# Переместим его туда
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Включим warning для Kaggle tool startup
!chmod 600 ~/.kaggle/kaggle.json

In [61]:
# Выведем список файлов соревнования advanced-dls-spring-2021
!kaggle competitions files advanced-dls-spring-2021

name             size  creationDate         
--------------  -----  -------------------  
train.csv       648KB  2021-03-03 13:03:16  
submission.csv   14KB  2021-03-03 13:03:16  
test.csv        213KB  2021-03-03 13:03:16  


In [62]:
# Загрузим все данные по соревнованию с Kaggle на Colab
!kaggle competitions download -p /data/advanced-dls-spring-2021 --force advanced-dls-spring-2021

  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 34.2MB/s]
  0% 0.00/648k [00:00<?, ?B/s]
100% 648k/648k [00:00<00:00, 42.8MB/s]
  0% 0.00/14.4k [00:00<?, ?B/s]
100% 14.4k/14.4k [00:00<00:00, 14.9MB/s]


In [63]:
# Выведем список загруженных файлов
!ls /data/advanced-dls-spring-2021

submission.csv	test.csv  train.csv


## Шаг 3. Загрузить начальные данные и посмотреть на них

In [64]:
data_dir = "/data/advanced-dls-spring-2021/"

In [65]:
# train.csv - файл с обучающей выборкой
# содержит колонки признаков и целевую переменную
train_data = pd.read_csv(data_dir + "train.csv")
train_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


In [66]:
# test.csv - файл с тестовой выборкой.
# содержит только колонки признаков
test_data = pd.read_csv(data_dir + "test.csv")
test_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,42,56.10,2386.85,Male,0,Yes,No,Yes,Yes,DSL,Yes,No,No,No,No,No,One year,No,Credit card (automatic)
1,29,98.50,3004.15,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
2,9,76.25,684.85,Male,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
3,67,74.00,4868.4,Female,0,Yes,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,Two year,No,Credit card (automatic)
4,71,20.10,1389.6,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic)


In [67]:
#submission.csv -  файла с предскзааниями
subm_data = pd.read_csv(data_dir + "submission.csv")
subm_data.head()

,Id,Churn
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
